# ИД23-1 Маслов АН
## Пример 4. Программа создает случайное количество строк в (кол-во ядер / 2) процессах, каждый процесс формирует результат в отдельном файле.
 

In [37]:
%%file rand_csv.py
import random
import time

def rand_csv(length, pos, time_start, time_end, output): #генератор случайно даты и числа
    csv_rows = []
    for i in range(length):
        stime = time.mktime(time.strptime(time_start, '%m/%d/%Y %I:%M %p'))
        etime = time.mktime(time.strptime(time_end, '%m/%d/%Y %I:%M %p'))
        ptime = stime + random.random() * (etime - stime)
        csv_rows.append([time.strftime('%y%m%d%H%M%S', time.localtime(ptime)), random.randint(1, 9999999), pos])
    output.put(csv_rows)

Overwriting rand_csv.py


In [38]:
import rand_csv

In [39]:
import multiprocessing as mp
import random
import pandas as pd

output = mp.Queue()

processes = [mp.Process(target=rand_csv.rand_csv, args=(random.randint(50000,500000), x, "1/1/2008 1:30 PM", "1/1/2025 4:50 AM", output)) \
             for x in range(int(mp.cpu_count() // 2))] #создаем (кол-во ядер / 2) процесса
for p in processes:
    p.start()

results = [output.get() for p in processes] #сохраняем результат

for p in processes:
    p.join()

pandas_result = []
for i in results:
    pandas_result = pandas_result + i
    resultdata = pd.DataFrame(pandas_result, columns=["datetime", "rand_number", "process_number"]) #создаем датафрейм
    resultdata['datetime'] = pd.to_datetime(resultdata['datetime'], format='%y%m%d%H%M%S') #форматируем для удобства
    resultdata = resultdata.sort_values('datetime') #сортируем по дате
    resultdata.to_csv("csv_result_"+str(resultdata["process_number"][0])+".csv", sep=';', index=False) #записываем результат в csv файлы
    pandas_result = []

## Написать функцию, которая считает среднее значение числового значения для каждого года по всем полученным файлам и возвращает результат в виде словаря. Не используйте параллельных вычислений. При реализации выделить функцию, которая объединяет результаты обработки отдельных файлов.

In [40]:
def avg_count(filename): #подсчёт среднего количества шагов по тэгу
    numbers_dictionary = {}
    data = pd.read_csv(filename, sep=';')
    for i, k in zip(data['datetime'], data['rand_number']):
        dates_clean = i[:4]
        if dates_clean not in numbers_dictionary:
            numbers_dictionary[dates_clean] = [1, k, 0]
        else:
            numbers_dictionary[dates_clean][0] = numbers_dictionary[dates_clean][0] + 1
            numbers_dictionary[dates_clean][1] = numbers_dictionary[dates_clean][1] + k
            numbers_dictionary[dates_clean][2] = numbers_dictionary[dates_clean][1] / numbers_dictionary[dates_clean][0]

    return numbers_dictionary


def numbers_count_merge(): #объединение средних значений по всем файлам
    numbers_avg = {}
    for i in range (0, 5):
        if numbers_avg == {}:
            numbers_avg = avg_count('csv_result_' + str(i) +'.csv')
        else:
            new_numbers = avg_count('csv_result_' + str(i) +'.csv')
            for k in numbers_avg.keys():
                if k not in numbers_avg:
                    numbers_avg[k] = numbers_avg[k]
                else:
                    numbers_avg[k][0] = numbers_avg[k][0] + new_numbers[k][0]
                    numbers_avg[k][1] = numbers_avg[k][1] + new_numbers[k][1]
                    numbers_avg[k][2] = numbers_avg[k][1] / numbers_avg[k][0]
                    
    print(pd.DataFrame(numbers_avg).rename(index={0: "Количество", 1: "Сумма", 2:"Среднее"}))

In [41]:
numbers_count_merge()

                    2008          2009          2010          2011  \
Количество  5.055300e+04  5.023200e+04  5.040300e+04  5.072100e+04   
Сумма       2.532717e+11  2.509488e+11  2.525448e+11  2.537438e+11   
Среднее     5.010024e+06  4.995795e+06  5.010512e+06  5.002736e+06   

                    2012          2013          2014          2015  \
Количество  5.100400e+04  5.048200e+04  5.017600e+04  5.070900e+04   
Сумма       2.557451e+11  2.528749e+11  2.503505e+11  2.526176e+11   
Среднее     5.014217e+06  5.009209e+06  4.989446e+06  4.981711e+06   

                    2016          2017          2018          2019  \
Количество  5.039100e+04  5.055900e+04  5.058800e+04  5.017100e+04   
Сумма       2.532631e+11  2.531656e+11  2.525771e+11  2.509289e+11   
Среднее     5.025959e+06  5.007330e+06  4.992827e+06  5.001473e+06   

                    2020          2021          2022          2023  \
Количество  5.101200e+04  5.053800e+04  5.043300e+04  5.021500e+04   
Сумма       2.552